In [1]:
# Import all the things

import json
import pandas as pd
import numpy as np
import wrangle
import acquire
from prepare import basic_clean, tokenize, lemmatize, stem, remove_stopwords, prep_string_data#, split_data

import re
from sklearn.feature_extraction.text import TfidfVectorizer

from wordcloud import WordCloud
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

import nltk

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [2]:
# use our own wrangle to get the data
# df = wrangle.get_npr_data()
# df.head()

### I'm interested in topic modeling on a story-level basis
### So I will need to groupby story_id_num

In [3]:
# df = df.sort_values(by=['story_id_num','utterance_order'])
# df.head()

In [4]:
# df = df[['story_id_num','utterance_order','title','lemmatized']]

In [5]:
# df_sample = df[:100]
# df_sample

In [6]:
# df_sample.shape

In [7]:
# pd.DataFrame(df_sample.groupby('title').apply(list).reset_index())

In [8]:
# df_sample.groupby(['story_id_num']).title.first()

In [9]:
# # group every story as an aggregation of all its utterances
# df_sample.groupby(['story_id_num']).lemmatized.agg(' '.join)#.iloc[0]

In [10]:
# # group every story as an aggregation of all its utterances and store in a DF
# stories_test = pd.DataFrame(df_sample.groupby(['story_id_num']).lemmatized.agg(' '.join))
# stories_test

In [11]:
# stories.iloc[0].values

### Run the above on the whole df

In [12]:
# df = df.sort_values(by=['story_id_num','utterance_order'])
# df.head()

In [13]:
# df.dtypes

In [14]:
# df.isna().sum()

In [15]:
# df = df.dropna()

#### well...it worked on the sample above
#### had to drop 2 nulls that existed in the lemmatized column

In [16]:
# # group every story as an aggregation of all its utterances and store in a DF
# stories = pd.DataFrame(df.groupby(['story_id_num']).lemmatized.agg(' '.join))
# stories

-  61,165 stories to work with here

In [17]:
# this attempt to get the word count for every story isn't working currently
# stories['word_count'] = df.lemmatized.apply(str.split).apply(len)
# stories.head()

In [18]:
# sia = nltk.sentiment.SentimentIntensityAnalyzer()
# stories['vader'] = stories.lemmatized.apply(lambda doc: sia.polarity_scores(doc)['compound'])
# stories.head()

In [19]:
# stories.to_csv('stories.csv')

In [20]:
stories = pd.read_csv('stories.csv')
stories.head()

,Unnamed: 0,story_id_num,lemmatized,vader,title
0,0,17,he is in cahoot with a foreign government to m...,0.9964,n.j. democrat on reported whistleblower compla...
1,1,18,"come with concrete plan , not beautiful speech...",0.9993,"ahead of climate summit, 2 views from cities i..."
2,2,19,if you stand outside the old dekalb county cou...,0.9974,"barred from removing confederate monument, cou..."
3,3,20,you may have spotted pink salt in your superma...,0.4950,pakistan's pink himalayan salt has become a ma...
4,4,21,"singing in foreign language . the other day , ...",0.9730,"10 years after haiti's earthquake, 'this music..."


In [21]:
stories = stories.drop(columns = ['Unnamed: 0'])

In [22]:
stories.shape

(61165, 4)

In [23]:
# len(df.groupby(['story_id_num']).title.first())

NameError: name 'df' is not defined

In [24]:
# stories['title'] = df.groupby(['story_id_num']).title.first().tolist()

In [25]:
stories.head()

,story_id_num,lemmatized,vader,title
0,17,he is in cahoot with a foreign government to m...,0.9964,n.j. democrat on reported whistleblower compla...
1,18,"come with concrete plan , not beautiful speech...",0.9993,"ahead of climate summit, 2 views from cities i..."
2,19,if you stand outside the old dekalb county cou...,0.9974,"barred from removing confederate monument, cou..."
3,20,you may have spotted pink salt in your superma...,0.4950,pakistan's pink himalayan salt has become a ma...
4,21,"singing in foreign language . the other day , ...",0.9730,"10 years after haiti's earthquake, 'this music..."


In [26]:
# stories.to_csv('stories.csv')

In [27]:
stories.vader.idxmax()

39

In [29]:
stories.lemmatized[39]

'looking ahead to our last day , many of our friend and colleague have been faced with a dilemma. well , how do you say goodbye ? for many of u , finding the right word can be difficult. and when inspiration fails and panic set in , well , we often turn to the long rack of greeting cards. so we ve asked former hallmark greeting card writer and creator of the youtube series greeting card emergency david elli dickerson. he ll join u in a moment. but what s the best goodbye card you ever got ? 800 989 8255. email u talknpr.org. david elli dickerson , with u here in studio 42. nice to have you back on the program . it s a lovely to be back , neal . and i have to say , broadly speaking , there seemed to be two categories. hey , congratulation , you re moving on , or , oh , i m so sorry . right , right. there s the you got fired and , you know , we re not going to see you , or you re going back to school. you re following your dreams. right. yeah . exactly. when you left hallmark , did anybo

In [30]:
stories.title[39]

'so hard to say goodbye: advice for farewell notes'

The first highest-sentiment article is about hallmark greeting cards

In [31]:
stories[stories.vader == 1]

,story_id_num,lemmatized,vader,title
39,620,"looking ahead to our last day , many of our fr...",1.0,so hard to say goodbye: advice for farewell notes
42,623,"this is talk of the nation. i m neal conan , i...",1.0,a look ahead and a farewell to the political j...
43,624,"this is talk of the nation. i m neal conan , i...",1.0,what changes after supreme court rulings on pr...
44,625,"from small country church , to the stage of th...",1.0,gospel legend mavis staples comes 'full circle'
141,898,this is talk of the nation. i m neal conan in ...,1.0,'blood &amp; beauty' breathes new life into th...
...,...,...,...,...
23796,84430,this is talk of the nation. i m neal conan in ...,1.0,state legislatures: agendas and budgets
23995,84850,time now for a story that is decidedly not a n...,1.0,encore: an elf questions santa's naughty or ni...
27366,90916,the political campaign are taking a break. the...,1.0,santa for president in 2016
30850,96419,"now we re going to take a break from news , fr...",1.0,listen: the not-so-true story of santa's naugh...


- Actually...there are 1345 stories with a max vader score

In [32]:
stories[stories.vader == -1]

,story_id_num,lemmatized,vader,title
749,2749,this is talk of the nation. i m neal conan in ...,-1.0,"the epidemiology of gun violence: race, region..."
5731,19881,this is talk of the nation. i m neal conan in ...,-1.0,how do you define a hate crime?
10730,55411,"this is talk of the nation. i m neal conan , i...",-1.0,understanding suicide terrorism and how to sto...
11827,58454,this is talk of the nation. im neal conan in w...,-1.0,deconstructing 'myths about suicide'
15177,66409,this is talk of the nation. i m neal conan in ...,-1.0,the legacy of bush's 'war on terror'
17237,71520,this is talk of the nation. i m neal conan in ...,-1.0,who carries out suicide bombings?
18838,75087,this is talk of the nation. i m neal conan in ...,-1.0,how do we define terrorism?
21189,79688,"this is talk of the nation. i m neal conan , i...",-1.0,should the guantanamo prison be closed?
21287,79858,"from npr news in washington , dc , i m neal co...",-1.0,'war of nerves': a history of chemical weapons


- But only 9 stories with the lowest vader score, including...

In [33]:
stories.lemmatized[749]

'this is talk of the nation. i m neal conan in washington. data from the center for disease control and prevention come to some remarkable conclusion about gun death , race , sex and region. briefly put , white people who die by gunfire are much more likely to be male , rural and suicide black people much more likely to be urban , victim of homicide and again , male . we ll explore the analysis , which wa featured in the washington post. we d also like to hear about your experiences. how ha gun violence affected you , your family and your neighborhood ? 800 989 8255. email u , talknpr.org. you can also join the conversation on our website. that s at npr.org. click on talk of the nation . later in the program , the coast guard and the northwest passage. but first gun death in america. dan keating is data editor and reporter with the washington post and join u now from studio at that newspaper. good to have you on talk of the nation today . good afternoon . and i understand of course , y

In [34]:
stories.title[749]

'the epidemiology of gun violence: race, region and policy'

- A story aboout "gun death , race , sex and region"

In [35]:
darkest_stories = stories[stories.vader == -1]
darkest_stories

,story_id_num,lemmatized,vader,title
749,2749,this is talk of the nation. i m neal conan in ...,-1.0,"the epidemiology of gun violence: race, region..."
5731,19881,this is talk of the nation. i m neal conan in ...,-1.0,how do you define a hate crime?
10730,55411,"this is talk of the nation. i m neal conan , i...",-1.0,understanding suicide terrorism and how to sto...
11827,58454,this is talk of the nation. im neal conan in w...,-1.0,deconstructing 'myths about suicide'
15177,66409,this is talk of the nation. i m neal conan in ...,-1.0,the legacy of bush's 'war on terror'
17237,71520,this is talk of the nation. i m neal conan in ...,-1.0,who carries out suicide bombings?
18838,75087,this is talk of the nation. i m neal conan in ...,-1.0,how do we define terrorism?
21189,79688,"this is talk of the nation. i m neal conan , i...",-1.0,should the guantanamo prison be closed?
21287,79858,"from npr news in washington , dc , i m neal co...",-1.0,'war of nerves': a history of chemical weapons


In [37]:
for story in darkest_stories.title:
    print(story)

the epidemiology of gun violence: race, region and policy
how do you define a hate crime?
understanding suicide terrorism and how to stop it
deconstructing 'myths about suicide'
the legacy of bush's 'war on terror'
who carries out suicide bombings?
how do we define terrorism?
should the guantanamo prison be closed?
'war of nerves': a history of chemical weapons


- Here's a list of the stories with the lowest possible sentiment score

In [38]:
brightest_stories = stories[stories.vader == 1]
brightest_stories.head(1)

,story_id_num,lemmatized,vader,title
39,620,"looking ahead to our last day , many of our fr...",1.0,so hard to say goodbye: advice for farewell notes


In [43]:
brightest_stories_title_list = []
for story in brightest_stories.title:
    brightest_stories_title_list.append(story)

In [44]:
# brightest_stories_title_list = pd.DataFrame(brightest_stories_title_list)
brightest_stories_title_list[:30]

['so hard to say goodbye: advice for farewell notes',
 'a look ahead and a farewell to the political junkie',
 'what changes after supreme court rulings on prop 8 and doma',
 "gospel legend mavis staples comes 'full circle'",
 "'blood &amp; beauty' breathes new life into the borgias",
 'the penultimate edition of the political junkie',
 'will work for free? the future of the unpaid internship',
 "india.arie returns, with an eye toward a new 'songversation'",
 "from 'casablanca' to 'e.t. the extra-terrestrial': famous farewells in film",
 'inside the cel: behind the scenes with animators',
 'a look ahead to the future of the gop',
 'the scifri book club takes a hike',
 'the role of the attorney general throughout history',
 'cartoonist lynda barry helps college students tap innate creativity',
 'looking ahead with the wonders of krulwich',
 'the promise and limitations of telemedicine',
 "iowa's open senate seat and rumbles about 2016",
 'hospital costs go public: what changes in health

-  At first glance, these brightest stories seem to mostly belong to a 'human interest' typee news category

In [46]:
stories.lemmatized[stories.title == "iowa's open senate seat and rumbles about 2016"].values

array(['this is talk of the nation. i m neal conan in washington. in the end , it s not close in south carolina. poll show it could be in massachusetts. and the president study up on korean culture. it s wednesday and time for a ... gangnam style ... edition of the political junkie . there you go again . when i hear your new idea , i m reminded of that ad where s the beef ? extremism in the defense of liberty is no vice . senator , you re no jack kennedy . you don t have nixon to kick around anymore . lipstick . oops . but i m the decider . every wednesday , political junkie ken rudin join u to review the week in politics. in south carolina , former governor mark sanford rise from his political grave. new jersey governor chris christie becomes a smaller man. the heritage foundation tally on immigration reform draw fire from both side of the aisle. house republican ask what secretary clinton knew and when she knew it. and new poll show cuccinelli up on mcauliffe in virginia . in a few m

- One oddball example in the bright stories, unusually about politics

## Below begins my attempt to use gensim for LDA topic modeling

In [47]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=False))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = stories.lemmatized.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brianclements/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['cahoot', 'foreign', 'government', 'manipulate', 'outcome', 'election', 'comment', 'democratic', 'presidential', 'candidate', 'california', 'senator', 'kamala', 'harris', 'impeachable', 'close', 'call', 'hawaii', 'senator', 'brian', 'schatz', 'new', 'york', 'congresswoman', 'alexandria', 'ocasio', 'cortez', 'point', 'beyond', 'constitutional']


In [48]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 3), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 4)]


In [49]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.023*"wa" + 0.012*"know" + 0.009*"think" + 0.008*"well" + 0.007*"one" + '
  '0.007*"going" + 0.006*"people" + 0.006*"like" + 0.005*"time" + 0.005*"ha"'),
 (1,
  '0.014*"wa" + 0.012*"know" + 0.012*"people" + 0.008*"think" + 0.007*"one" + '
  '0.007*"say" + 0.007*"like" + 0.006*"ha" + 0.005*"well" + 0.005*"going"'),
 (2,
  '0.011*"wa" + 0.009*"say" + 0.008*"know" + 0.007*"think" + 0.007*"ha" + '
  '0.007*"well" + 0.007*"people" + 0.006*"like" + 0.006*"one" + 0.006*"year"'),
 (3,
  '0.010*"wa" + 0.008*"know" + 0.008*"people" + 0.007*"say" + 0.006*"ha" + '
  '0.006*"like" + 0.006*"well" + 0.005*"think" + 0.005*"would" + 0.005*"year"'),
 (4,
  '0.011*"wa" + 0.009*"one" + 0.008*"know" + 0.008*"people" + 0.006*"ha" + '
  '0.006*"say" + 0.006*"like" + 0.006*"think" + 0.005*"get" + 0.005*"would"'),
 (5,
  '0.012*"wa" + 0.011*"know" + 0.009*"think" + 0.008*"people" + 0.007*"say" + '
  '0.007*"one" + 0.006*"like" + 0.006*"going" + 0.006*"get" + 0.006*"ha"'),
 (6,
  '0.012*"wa" + 0.008*"k

In [57]:
# import pyLDAvis.gensim
import os
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
# LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+ str(num_topics)+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
#     LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
#     with open(LDAvis_data_filepath, 'wb') as f:
#         pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
# with open(LDAvis_data_filepath, 'rb') as f:
#     LDAvis_prepared = pickle.load(f)
# pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.004740  0.013237       1        1  17.670618
0     -0.008617 -0.004612       2        1  14.671827
4     -0.008079 -0.007064       3        1  10.490239
1     -0.001445 -0.002192       4        1   9.853490
6      0.012266 -0.003230       5        1   9.558336
3      0.010991  0.000681       6        1   9.310276
7      0.006721  0.001301       7        1   7.809881
2     -0.004596  0.004428       8        1   7.394109
8     -0.003360 -0.002061       9        1   7.329735
9      0.000860 -0.000488      10        1   5.911489, topic_info=       Term           Freq          Total Category  logprob  loglift
224      wa  475907.000000  475907.000000  Default  30.0000  30.0000
114    know  345722.000000  345722.000000  Default  29.0000  29.0000
150  people  260984.000000  260984.000000  Default  28.0000  28.0000
86       ha  203508.000000  203508.000000  Default  27.0000  27.0000
178     say  224140.000000  224140.000000  Default  26.0000  26.0000
..      ...            ...            ...      ...      ...      ...
83    going    8715.216248  186605.578249  Topic10  -5.4285  -0.2357
437  really    7331.333408  140350.603871  Topic10  -5.6014  -0.1237
82      get    7698.561591  165338.354719  Topic10  -5.5525  -0.2387
202   think    8981.022437  257090.759011  Topic10  -5.3985  -0.5260
442   right    7036.112409  145986.412761  Topic10  -5.6425  -0.2042

[941 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
21922      1  0.126737     abbas
21922      2  0.033869     abbas
21922      3  0.049165     abbas
21922      4  0.036055     abbas
21922      5  0.585614     abbas
...      ...       ...       ...
32445      6  0.064109  zubaydah
32445      7  0.128218  zubaydah
32445      8  0.016027  zubaydah
32445      9  0.384654  zubaydah
32445     10  0.032054  zubaydah

[4489 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 5, 2, 7, 4, 8, 3, 9, 10])

## The same...but running it on the titles instead of the corpi

In [58]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=False))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = stories.title.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brianclements/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['democrat', 'reported', 'whistleblower', 'complaint', 'trump']


In [59]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]


In [60]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.015*"new" + 0.005*"year" + 0.005*"orleans" + 0.005*"one" + '
  '0.004*"program" + 0.004*"time" + 0.004*"school" + 0.004*"home" + '
  '0.004*"black" + 0.003*"class"'),
 (1,
  '0.008*"program" + 0.007*"class" + 0.006*"iran" + 0.005*"news" + '
  '0.004*"years" + 0.004*"report" + 0.004*"brief" + 0.004*"president" + '
  '0.003*"trump" + 0.003*"crisis"'),
 (2,
  '0.015*"new" + 0.012*"obama" + 0.006*"may" + 0.005*"bush" + 0.005*"trump" + '
  '0.005*"war" + 0.005*"senate" + 0.004*"democrats" + 0.004*"gop" + '
  '0.004*"debate"'),
 (3,
  '0.015*"program" + 0.015*"class" + 0.005*"news" + 0.004*"new" + '
  '0.004*"public" + 0.004*"radio" + 0.004*"nuclear" + 0.004*"health" + '
  '0.003*"years" + 0.003*"may"'),
 (4,
  '0.007*"court" + 0.006*"iraq" + 0.005*"new" + 0.005*"trump" + 0.004*"news" + '
  '0.004*"war" + 0.004*"program" + 0.004*"class" + 0.004*"supreme" + '
  '0.004*"american"'),
 (5,
  '0.010*"new" + 0.006*"obama" + 0.005*"class" + 0.005*"program" + '
  '0.005*"letters" + 0.004*"

In [61]:
# import pyLDAvis.gensim
import os
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
# LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+ str(num_topics)+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
#     LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
#     with open(LDAvis_data_filepath, 'wb') as f:
#         pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
# with open(LDAvis_data_filepath, 'rb') as f:
#     LDAvis_prepared = pickle.load(f)
# pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.041109 -0.039615       1        1  10.910202
4      0.027630 -0.062811       2        1  10.304528
2      0.047923  0.069807       3        1  10.301583
5     -0.007654  0.016205       4        1  10.200696
6      0.096603 -0.026209       5        1  10.114892
0     -0.122651  0.030238       6        1   9.777118
9     -0.016579 -0.036736       7        1   9.755298
1     -0.008247 -0.059022       8        1   9.730782
8      0.018530  0.110192       9        1   9.696483
7      0.005555 -0.002049      10        1   9.208420, topic_info=          Term         Freq        Total Category  logprob  loglift
2067     obama  1552.000000  1552.000000  Default  30.0000  30.0000
266        new  2760.000000  2760.000000  Default  29.0000  29.0000
106    program  1948.000000  1948.000000  Default  28.0000  28.0000
103      class  1841.000000  1841.000000  Default  27.0000  27.0000
92    politics   631.000000   631.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
2067     obama    96.401541  1552.973396  Topic10  -5.8348  -0.3944
319      radio    77.407628   622.153534  Topic10  -6.0543   0.3010
1241      high    71.045191   522.122069  Topic10  -6.1400   0.3905
2180  congress    68.068413   425.669546  Topic10  -6.1828   0.5519
601        get    67.881839   450.325133  Topic10  -6.1856   0.4929

[901 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
8762       7  0.873599  abdullah
642        1  0.037500  abortion
642        2  0.435000  abortion
642        3  0.030000  abortion
642        4  0.030000  abortion
...      ...       ...       ...
3459       4  0.471617     youth
3459       5  0.083227     youth
3459       6  0.110969     youth
3459       7  0.124840     youth
18140     10  0.961135   zarqawi

[3380 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 3, 6, 7, 1, 10, 2, 9, 8])

## Evaluate? (this is running on the 'title' topic modeling)

In [63]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [64]:
# NLTK Stop words
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [66]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['report', 'complaint']]


In [67]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1), (1, 1)]]


In [68]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [69]:
# from pprint import pprint
# # Print the Keyword in the 10 topics
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [70]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Coherence Score:  0.3981690722857686


In [71]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  0%|          | 0/540 [02:42<?, ?it/s]

 26%|██▋       | 142/540 [1:08:07<3:23:50, 30.73s/it]


 53%|█████▎    | 284/540 [8:07:43<2:15:11, 31.69s/it]


 78%|███████▊  | 422/540 [9:34:51<1:34:38, 48.12s/it]

In [ ]:
# use the above-selected parameters to train the model

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [ ]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared